In [61]:
import pandas as pd
import os
from os import listdir
from os.path import isfile, join

In [22]:
path_base = r"C:\Users\DellVostro\OneDrive\Documentos\XP\Projeto_aplicado\Enso\Dados_clima_RS"

In [19]:
def coleta_lista_nomes(path):
   arquivos = [f for f in listdir(path) if isfile(join(path, f))] 

   return arquivos[:5]

In [20]:
coleta_lista_nomes(path)

['INMET_S_RS_A801_PORTO ALEGRE_01-01-2001_A_31-12-2001.CSV',
 'INMET_S_RS_A802_RIO GRANDE_16-11-2001_A_31-12-2001.CSV',
 'INMET_S_RS_A803_SANTA MARIA_26-11-2001_A_31-12-2001.CSV',
 'INMET_S_RS_A804_SANTANA DO LIVRAMENTO_22-11-2001_A_31-12-2001.CSV',
 'INMET_S_RS_A805_SANTO AUGUSTO_05-12-2001_A_31-12-2001.CSV']

In [27]:
df = pd.read_csv(
                    caminho_base,
                    sep=";",
                    header=8,
                    encoding="iso-8859-1",
                    usecols=[0, 2]
                )

In [24]:
import os
import pandas as pd

def coleta_dados(path_base):
    dfs = []

    for root, dirs, files in os.walk(path_base):
        arquivos = [f for f in files if os.path.isfile(os.path.join(root, f))]
        primeiros_5 = sorted(arquivos)[:5]  # Ordena e seleciona os cinco primeiros

        for nome_arquivo in primeiros_5:
            caminho_base = os.path.join(root, nome_arquivo)
            try:
                df = pd.read_csv(
                    caminho_base,
                    sep=";",
                    header=8,
                    encoding="iso-8859-1",
                    usecols=[0, 2]
                )
                dfs.append(df)
            except Exception as e:
                print(f"Erro ao ler {caminho_base}: {e}")

    if dfs:
        return pd.concat(dfs, ignore_index=True)
    else:
        print("Nenhum arquivo válido foi encontrado.")
        return pd.DataFrame()



In [52]:
df = coleta_dados(path_base)

In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 976536 entries, 0 to 976535
Data columns (total 3 columns):
 #   Column                            Non-Null Count   Dtype 
---  ------                            --------------   ----- 
 0   DATA (YYYY-MM-DD)                 757416 non-null  object
 1   PRECIPITAÇÃO TOTAL, HORÁRIO (mm)  968492 non-null  object
 2   Data                              219120 non-null  object
dtypes: object(3)
memory usage: 22.4+ MB


In [56]:
df["PRECIPITAÇÃO TOTAL, HORÁRIO (mm)"].replace({"-9999": "0"}, inplace = True)

C:\Users\DellVostro\AppData\Local\Temp\ipykernel_14720\547132121.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["PRECIPITAÇÃO TOTAL, HORÁRIO (mm)"].replace({"-9999": "0"}, inplace = True)


In [57]:
df["PRECIPITAÇÃO TOTAL, HORÁRIO (mm)"] = df["PRECIPITAÇÃO TOTAL, HORÁRIO (mm)"].str.replace(",", ".").astype(float)

In [58]:
df['Ano'] = df['DATA (YYYY-MM-DD)'].str[:4]

In [60]:
from datetime import datetime

In [62]:
df["DATA (YYYY-MM-DD)"] =  pd.to_datetime(df["DATA (YYYY-MM-DD)"])

In [66]:
df["Ano"].unique()

array(['2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016',
       '2017', '2018', nan], dtype=object)

In [63]:
df["DATA (YYYY-MM-DD)"]

0        2001-01-01
1        2001-01-01
2        2001-01-01
3        2001-01-01
4        2001-01-01
            ...    
976531          NaT
976532          NaT
976533          NaT
976534          NaT
976535          NaT
Name: DATA (YYYY-MM-DD), Length: 976536, dtype: datetime64[ns]